In [ ]:
from flask import Flask, render_template, request, send_file, redirect, url_for, session, jsonify
from flask_qrcode import QRcode as FlaskQRcode
from flask_mysqldb import MySQL
import qrcode
from pyzbar.pyzbar import decode  # Added import for QR code decoding
from io import BytesIO
import os

app = Flask(__name__, template_folder='temp')
app.secret_key = 'your_secret_key'  # Set a secret key for sessions

# MySQL configurations
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'M@hev!$h2003:'
app.config['MYSQL_DB'] = 'python'

# QR code extension with a different name to avoid conflict
flask_qrcode = FlaskQRcode(app)

# Create the MySQL object
mysql = MySQL(app)

# Function to generate and store QR code for a user
def generate_qr_code(username):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )

    # Generate QR code data
    qr.add_data(f'Username:{username}')
    qr.make(fit=True)

    # Generate QR code image
    img = qr.make_image(fill_color="black", back_color="white")

    # Save QR code image to BytesIO
    img_bytes = BytesIO()
    img.save(img_bytes, format='PNG')
    img_bytes.seek(0)

    # Save QR code image to a file
    folder_path = 'qr_Codes'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_path = os.path.join(folder_path, f'qr_code_{username}.png')
    img.save(file_path)

    return img_bytes, file_path

# Function to check if a user with the same username already exists
def user_exists(username):
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM users WHERE username = %s", (username,))
    existing_user = cur.fetchone()
    cur.close()
    return existing_user is not None

# Route for downloading the QR code for a specific user
@app.route('/get_qr_code/<username>')
def get_qr_code(username):
    img_bytes, file_path = generate_qr_code(username)
    return send_file(img_bytes, mimetype='image/png', download_name=f'qr_code_{username}.png')

# Route for the registration page
@app.route('/')
def registration_page():
    return render_template('registrt.html')

from werkzeug.utils import secure_filename

# Set the upload folder and allowed extensions
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Route for handling registration form submission
@app.route('/register', methods=['POST'])
def register_user():
    # Your registration logic here
    username = request.form.get('username')
    password = request.form.get('password')
    email = request.form.get('email')

    # Check if the user with the same username already exists
    if user_exists(username):
        session['message'] = 'User already exists. Please choose a different username.'
        return redirect(url_for('registration_page'))

    # Handle photo upload
    if 'photo' not in request.files:
        session['message'] = 'No file part'
        return redirect(request.url)

    photo = request.files['photo']

    if photo.filename == '':
        session['message'] = 'No selected file'
        return redirect(request.url)

    if photo and allowed_file(photo.filename):
        filename = secure_filename(photo.filename)
        photo.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

    # Insert user data into the MySQL database
    cur = mysql.connection.cursor()
    cur.execute('INSERT INTO users (username, password, email, photo_filename) VALUES (%s, %s, %s, %s)',
                (username, password, email, filename if photo else None))
    mysql.connection.commit()
    cur.close()

    # Generate and store QR code for the user
    img_bytes, file_path = generate_qr_code(username)

    session['message'] = 'Registration successful!'
    return send_file(img_bytes, mimetype='image/png', download_name=f'qr_code_{username}.png')

# New route for QR code decoding
@app.route('/decode_qr_code', methods=['POST'])
def decode_qr_code():
    # Get the QR code image from the request
    qr_image = request.files['qr_image']

    # Decode the QR code
    decoded_data = decode(qr_image)

    if decoded_data:
        # Assuming the QR code contains the username
        username = decoded_data[0].data.decode('utf-8').replace('USERNAME : ', '').strip()

        # Fetch user data from the database
        user_data = fetch_user_data(username)

        if user_data:
            return jsonify({'status': 'success', 'user_data': user_data})
        else:
            return jsonify({'status': 'error', 'message': 'User not found'})
    else:
        return jsonify({'status': 'error', 'message': 'QR code decoding failed'})

# Function to fetch user data from the database
def fetch_user_data(username):
    cur = mysql.connection.cursor()
    cur.execute("SELECT * FROM users WHERE username = %s", (username, ))
    user_data = cur.fetchone()
    cur.close()
    return user_data

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
